In [8]:
import pandas as pd
import requests
from keys import API_key_openweather

### 1. Connection with my local host in mySQL

In [ ]:
#from keys import mySQL_pass

schema="gans"   # name of the database you want to use here
host="localhost"        # to connect to your local server
user="root"
password=mySQL_pass # your password!!!!
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

### 2. Connection with AWS relational DB

In [3]:
from con_aws_to_sql import aws_pass

schema="gans"   # name of the database you want to use here
host='wbs-project4-db.c1iajchyyr27.eu-north-1.rds.amazonaws.com'
username = 'admin'
port = 3306
password = aws_pass #Amazon RDS in mySQL
con = f'mysql+pymysql://{username}:{password}@{host}:{port}/{schema}'

### 3. Creating the table 'cities' which is the foundation of the schema dataframes
### You don't need to create this again. once done, it is DONE!!!

In [ ]:
import pandas as pd
import requests
from keys import API_key_openweather

# Limit the number of cities to fetch data for each API call
limit = 5

# List of cities to fetch data for
cities = ['Berlin', 'Munich', 'Frankfurt', 'London', 'Copenhagen']

# Lists to store city data
name = []
latitude = []
longitude = []
country = []

# Lists to store weather data (to be filled later)
temp = []
temp_min = []
temp_max = []
humidity = []
main_weather = []
description = []
wind_speed = []
visibility = []
datetime = []

# Fetch geographical data for each city
for city in cities:
    # API request to get city data (name, lat, lon, country) from OpenWeather API
    openweather_cities = requests.get(f'http://api.openweathermap.org/geo/1.0/direct?q={city}&limit={limit}&appid={API_key_openweather}')
    cities_json = openweather_cities.json()

    # Append fetched data to corresponding lists
    name.append(cities_json[0]['name'])
    latitude.append(cities_json[0]['lat'])
    longitude.append(cities_json[0]['lon'])
    country.append(cities_json[0]['country'])

# Create a DataFrame to store location data for each city
location_data_per_city = pd.DataFrame(list(zip(name, country, latitude, longitude)), columns=['city_name', 'country_code', 'latitude', 'longitude'])

----- This is designed to send the dataframe 'cities' into mySQL Workbench table 'cities' -----

In [ ]:
location_data_per_city.to_sql("cities",
                 if_exists="append",
                 con=con,
                 index=False
                )

---- To read 'cities' table from mysql -----

In [4]:
cities = pd.read_sql("cities", con=con)

In [5]:
cities

,city_id,city_name,country_code,latitude,longitude
0,1,Berlin,DE,52.5170,13.388900
1,2,Munich,DE,48.1371,11.575400
2,3,Frankfurt,DE,50.1106,8.682090
3,4,London,GB,51.5073,-0.127647
4,5,Copenhagen,DK,55.6867,12.570100


### 4. The Function 'forecast'

In [6]:
def forecast(cities):
    """
    Fetches 5-day weather forecast for given cities.
    
    Args:
    cities (pd.DataFrame): DataFrame containing city data with latitude and longitude.
    
    Returns:
    pd.DataFrame: DataFrame containing weather forecast information.
    """
    
    # Lists to store forecast data
    city_id = []
    temp = []
    temp_min = []
    temp_max = []
    humidity = []
    main_weather = []
    description = []
    wind_speed = []
    visibility = []
    datetime = []

    # Iterate over each city to fetch weather forecast
    for i, row in cities.iterrows():
        # API request to get 5-day weather forecast data from OpenWeather API
        openweather_5days = requests.get(f'http://api.openweathermap.org/data/2.5/forecast?lat={row["latitude"]}&lon={row["longitude"]}&appid={API_key_openweather}')
        weather_json = openweather_5days.json()

        # Iterate over the first 8 time intervals (next 24 hours)
        for i, time in enumerate(weather_json['list'][:8]):
            city_id.append(row['city_id'])
            temp.append(time['main']['temp'])
            temp_min.append(time['main']['temp_min'])
            temp_max.append(time['main']['temp_max'])
            humidity.append(time['main']['humidity'])
            main_weather.append(time['weather'][0]['main'])
            description.append(time['weather'][0]['description'])
            wind_speed.append(time['wind']['speed'])
            visibility.append(time['visibility'])
            datetime.append(time['dt_txt'])

    # Create DataFrame from collected forecast data
    return pd.DataFrame(list(zip(city_id, temp, temp_min, temp_max, humidity, main_weather, description, wind_speed, visibility, datetime)),
                        columns=['city_id', 'temperature', 'min', 'max', 'humidity', 'weather', 'description', 'wind_speed', 'visibility', 'forecast_time'])

In [9]:
forecast(cities)

,city_id,temperature,min,max,humidity,weather,description,wind_speed,visibility,forecast_time
0,1,274.03,273.44,274.03,94,Snow,light snow,1.76,26,2023-12-06 18:00:00
1,1,273.57,273.19,273.57,97,Clouds,overcast clouds,1.41,108,2023-12-06 21:00:00
2,1,273.28,273.28,273.28,99,Clouds,overcast clouds,1.37,621,2023-12-07 00:00:00
3,1,273.35,273.35,273.35,99,Clouds,overcast clouds,1.42,60,2023-12-07 03:00:00
4,1,273.37,273.37,273.37,99,Clouds,overcast clouds,1.40,39,2023-12-07 06:00:00
5,1,273.54,273.54,273.54,100,Clouds,overcast clouds,1.49,39,2023-12-07 09:00:00
6,1,274.02,274.02,274.02,100,Clouds,overcast clouds,1.64,52,2023-12-07 12:00:00
7,1,273.69,273.69,273.69,100,Clouds,overcast clouds,1.22,40,2023-12-07 15:00:00
8,2,274.61,272.55,274.61,90,Clouds,overcast clouds,3.16,10000,2023-12-06 18:00:00
9,2,273.65,272.66,273.65,94,Snow,light snow,2.69,10000,2023-12-06 21:00:00


### 4. This will send dataframe created with the function to mySQL Workbench

In [ ]:
forecast(cities).to_sql('forecast',if_exists='append',con=con,index=False)